# Implementación Caso 1 CVRP

In [ ]:
import math
import time
from pathlib import Path
import pandas as pd
import pyomo.environ as pyo

start_time = time.time()

clients_df = pd.read_csv("clients.csv")
depots_df    = pd.read_csv("depots.csv")
veh_df      = pd.read_csv("vehicles.csv")
params_df   = pd.read_csv("parameters_urban.csv")


#  único depósito (CD01) en la fila 0
depot_row = depots_df.iloc[0]
depot_external_id = str(depot_row.get("DepotID", "CD01"))
depot_lat = float(depot_row["Latitude"])
depot_lon = float(depot_row["Longitude"])

# Clientes (IDs tal como vienen, convertidos a str)
clients_external = clients_df["ClientID"].astype(str).tolist()

# Map internal index -> external id 
idx_to_external = {0: depot_external_id}
external_to_idx = {depot_external_id: 0}
for i, cid in enumerate(clients_external, start=1):
    cid_s = str(cid)
    idx_to_external[i] = cid_s
    external_to_idx[cid_s] = i

# Coordenadas por índice interno
coords = {0: (depot_lat, depot_lon)}
for _, r in clients_df.iterrows():
    cid = str(r["ClientID"])
    coords[external_to_idx[cid]] = (float(r["Latitude"]), float(r["Longitude"]))

# Demandas por índice interno
demand = {}
for _, r in clients_df.iterrows():
    cid = str(r["ClientID"])
    demand[external_to_idx[cid]] = float(r["Demand"])

# Vehículos (IDs externos, strings)
V_list = veh_df["VehicleID"].astype(str).tolist()
# capacidades y autonomías por vehicle id
rv = {}
tv = {}
ev = {}
for _, r in veh_df.iterrows():
    vid = str(r["VehicleID"])
    rv[vid] = float(r.get("Capacity", 0.0))
    tv[vid] = float(r.get("Range", 0.0))
    ev[vid] = float(r.get("FuelEfficiency", params_df.loc[params_df["Parameter"]=="fuel_efficiency_typical","Value"].iloc[0]
                          if ("fuel_efficiency_typical" in params_df["Parameter"].values) else 10.0))

# parámetros globales
def pval(name, default):
    s = params_df.loc[params_df["Parameter"]==name, "Value"]
    return float(s.iloc[0]) if not s.empty else default

pf = pval("fuel_price", 12000.0)            # COP / litro
fuel_eff_typ = pval("fuel_efficiency_typical", 10.0)  # km/l 
co = 50000.0   # costo fijo por vehículo (COP)

# Si algún vehículo no tiene ev definido, asignar típico
for vid in V_list:
    if vid not in ev or ev[vid] <= 0:
        ev[vid] = fuel_eff_typ

# nodos internos
DEPOT_IDX = 0
CLIENT_INDICES = list(range(1, 1 + len(clients_external)))
NODOS = [DEPOT_IDX] + CLIENT_INDICES

def haversine_km(lat1, lon1, lat2, lon2):
    R = 6371.0
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)
    a = math.sin(dphi / 2.0)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c

# distancia matriz
w = {}
for i in NODOS:
    for j in NODOS:
        if i == j:
            w[i,j] = 0.0
        else:
            w[i,j] = haversine_km(coords[i][0], coords[i][1], coords[j][0], coords[j][1])

# modelo
model = pyo.ConcreteModel()

model.N = pyo.Set(initialize=NODOS)
model.C = pyo.Set(initialize=CLIENT_INDICES)
model.V = pyo.Set(initialize=V_list)
model.D = pyo.Set(initialize=[DEPOT_IDX])

# Params
model.w = pyo.Param(model.N, model.N, initialize=w, within=pyo.NonNegativeReals)
model.q = pyo.Param(model.C, initialize={c: demand[c] for c in CLIENT_INDICES}, within=pyo.NonNegativeReals)
model.r = pyo.Param(model.V, initialize=rv, within=pyo.NonNegativeReals)
model.tau = pyo.Param(model.V, initialize=tv, within=pyo.NonNegativeReals)
model.e = pyo.Param(model.V, initialize=ev, within=pyo.PositiveReals)

model.co = pyo.Param(initialize=co)
model.pf = pyo.Param(initialize=pf)

# costo por arco (solo combustible en caso base)
def s_init(m, i, j, v):
    return (m.pf / m.e[v]) * m.w[i,j]
model.s = pyo.Param(model.N, model.N, model.V, initialize=s_init, within=pyo.NonNegativeReals)

# variables
model.x = pyo.Var(model.N, model.N, model.V, domain=pyo.Binary)   # arco (i,j) usado por v
model.y = pyo.Var(model.V, domain=pyo.Binary)                     # vehiculo activo
model.z = pyo.Var(model.C, model.V, domain=pyo.Binary)            # veh v visita cliente c
# load  (client,vehicle) cantidad acumulada luego de servir el cliente c por v
max_cap = max(rv.values()) if rv else 1.0
model.u = pyo.Var(model.C, model.V, domain=pyo.NonNegativeReals, bounds=(0, max_cap))
# restricciones
def no_self_arcs(m, i, v):
    return m.x[i,i,v] == 0
model.no_self_arcs = pyo.Constraint(model.N, model.V, rule=no_self_arcs)

def link_x_y(m, i, j, v):
    return m.x[i,j,v] <= m.y[v]
model.link_x_y = pyo.Constraint(model.N, model.N, model.V, rule=link_x_y)

def entry_def(m, c, v):
    return sum(m.x[i,c,v] for i in m.N if i != c) == m.z[c,v]
model.entry_def = pyo.Constraint(model.C, model.V, rule=entry_def)

def exit_def(m, c, v):
    return sum(m.x[c,j,v] for j in m.N if j != c) == m.z[c,v]
model.exit_def = pyo.Constraint(model.C, model.V, rule=exit_def)

def cover_once(m, c):
    return sum(m.z[c,v] for v in m.V) == 1
model.cover_once = pyo.Constraint(model.C, rule=cover_once)

def capacity_rule(m, v):
    return sum(m.q[c] * m.z[c,v] for c in m.C) <= m.r[v] * m.y[v]
model.capacity = pyo.Constraint(model.V, rule=capacity_rule)

def depart_rule(m, v):
    return sum(m.x[DEPOT_IDX, j, v] for j in m.C) == m.y[v]
model.depart = pyo.Constraint(model.V, rule=depart_rule)

def return_rule(m, v):
    return sum(m.x[i, DEPOT_IDX, v] for i in m.C) == m.y[v]
model.return_depot = pyo.Constraint(model.V, rule=return_rule)

def autonomy(m, v):
    return sum(m.x[i,j,v] * m.w[i,j] for i in m.N for j in m.N if i != j) <= m.tau[v] * m.y[v]
model.autonomy = pyo.Constraint(model.V, rule=autonomy)

def u_lower(m, c, v):
    return m.u[c,v] >= m.q[c] * m.z[c,v]
model.u_low = pyo.Constraint(model.C, model.V, rule=u_lower)

def u_upper(m, c, v):
    return m.u[c,v] <= m.r[v] * m.z[c,v]
model.u_up = pyo.Constraint(model.C, model.V, rule=u_upper)

def mtz_rule(m, c, j, v):
    if c == j:
        return pyo.Constraint.Skip
    return m.u[c,v] - m.u[j,v] + m.r[v] * m.x[c,j,v] <= m.r[v] - m.q[j] * m.z[j,v]
model.mtz = pyo.Constraint(model.C, model.C, model.V, rule=mtz_rule)

# f objetivo
def obj_rule(m):
    var_cost = sum(m.s[i,j,v] * m.x[i,j,v] for i in m.N for j in m.N for v in m.V)
    fixed_cost = sum(m.co * m.y[v] for v in m.V)
    return var_cost + fixed_cost
model.OBJ = pyo.Objective(rule=obj_rule, sense=pyo.minimize)


solver = pyo.SolverFactory('glpk')
solver.options["tmlim"] = 300  

res = solver.solve(model, tee=True)

print("Solver status:", res.solver.status, "Termination:", res.solver.termination_condition)


def reconstruct_routes(model):
    routes = {}
    for v in model.V:
        if pyo.value(model.y[v]) is None or pyo.value(model.y[v]) < 0.5:
            continue
        route = [DEPOT_IDX]
        current = DEPOT_IDX
        visited = []
        steps_limit = len(NODOS) * 3
        for _ in range(steps_limit):
            found = False
            for j in NODOS:
                if j == current: 
                    continue
                xv = pyo.value(model.x[current, j, v])
                if xv is not None and xv > 0.5:
                    route.append(j)
                    if j != DEPOT_IDX:
                        visited.append(j)
                    current = j
                    found = True
                    break
            if not found:
                break
            if current == DEPOT_IDX:
                break
        if route[-1] != DEPOT_IDX:
            route.append(DEPOT_IDX)
        #  metricas
        dist = sum(w[route[i], route[i+1]] for i in range(len(route)-1))
        fuel_cost = sum((w[route[i], route[i+1]] / ev[v]) * pf for i in range(len(route)-1))
        load = sum(int(demand[c]) for c in visited) if visited else 0
        routes[v] = {
            "route": route,
            "visited": visited,
            "distance": dist,
            "fuel_cost": fuel_cost,
            "load": load
        }
    return routes

routes_veh = reconstruct_routes(model)

# Exportar verificacion_caso1.csv 
rows = []
for v, info in routes_veh.items():
    seq_external = []
    for idx in info["route"]:
        seq_external.append(idx_to_external.get(idx, f"C{idx}"))
    route_seq = " - ".join(seq_external)
    demands_satisfied = " - ".join(str(int(demand[c])) for c in info["visited"]) if info["visited"] else ""
    rows.append({
        "VehicleId": v,
        "LoadCap": int(rv.get(v,0)),
        "FuelCap": round(tv.get(v,0) / ev.get(v, fuel_eff_typ), 2),  
        "RouteSequence": route_seq,
        "Municipalities": len(info["visited"]),
        "DemandSatisfied": demands_satisfied,
        "InitialLoad": int(info["load"]),
        "InitialFuel": round(tv.get(v,0) / ev.get(v, fuel_eff_typ), 2),
        "Distance": round(info["distance"], 2),
        "Time": round(info["distance"] / 40.0 * 60.0, 2), 
        "TotalCost": int(round(info["fuel_cost"] + co))
    })

verif_df = pd.DataFrame(rows, columns=[
    "VehicleId","LoadCap","FuelCap","RouteSequence","Municipalities","DemandSatisfied",
    "InitialLoad","InitialFuel","Distance","Time","TotalCost"
])
if not verif_df.empty:
    verif_df.to_csv("verificacion_caso1.csv", index=False)
    print("Archivo verificacion_caso1.csv guardado.")
else:
    print("No se encontraron vehículos en la solución - no se guardó verificacion_caso1.csv")

# resumen
total_dist = sum(info["distance"] for info in routes_veh.values())
total_fuel_cost = sum(info["fuel_cost"] for info in routes_veh.values())
total_fixed = co * len(routes_veh)
total_cost = total_fuel_cost + total_fixed

print("\n--- Resumen ---")
print(f"Vehiculos usados: {len(routes_veh)} / {len(V_list)}")
print(f"Distancia total: {total_dist:.2f} km")
print(f"Combustible: {total_fuel_cost:,.0f} COP, Fijos: {total_fixed:,.0f} COP, Total: {total_cost:,.0f} COP")
print(f"Tiempo total script: {time.time() - start_time:.1f} s")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --tmlim 300 --write C:\Users\Andres\AppData\Local\Temp\tmpwtt0cvxl.glpk.raw
 --wglp C:\Users\Andres\AppData\Local\Temp\tmpq93visk6.glpk.glp --cpxlp C:\Users\Andres\AppData\Local\Temp\tmp2x8qm_qh.pyomo.lp
Reading problem data from 'C:\Users\Andres\AppData\Local\Temp\tmp2x8qm_qh.pyomo.lp'...
C:\Users\Andres\AppData\Local\Temp\tmp2x8qm_qh.pyomo.lp:85362: warning: lower bound of variable 'x4811' redefined
C:\Users\Andres\AppData\Local\Temp\tmp2x8qm_qh.pyomo.lp:85362: warning: upper bound of variable 'x4811' redefined
10440 rows, 5392 columns, 43832 non-zeros
5200 integer variables, all of which are binary
90562 lines were read
Writing problem data to 'C:\Users\Andres\AppData\Local\Temp\tmpq93visk6.glpk.glp'...
74507 lines were written
GLPK Integer Optimizer 5.0
10440 rows, 5392 columns, 43832 non-zeros
5200 integer variables, all of which are binary
Preprocessing...
10040 rows, 5192 columns, 43232 non-zeros
5000 in

In [6]:
import pandas as pd
import folium
import math

df = pd.read_csv("verificacion_caso1.csv")

all_nodes = set()
for seq in df["RouteSequence"]:
    nodes = [int(x.strip()) for x in seq.split("-")]
    all_nodes.update(nodes)

all_nodes = sorted(all_nodes)

def generate_fake_coordinates(nodes):
    coords = {}
    n = len(nodes)
    radius = 0.05 
    center_lat, center_lon = 4.0, -74.0  
    
    for i, node in enumerate(nodes):
        angle = 2 * math.pi * i / n
        lat = center_lat + radius * math.sin(angle)
        lon = center_lon + radius * math.cos(angle)
        coords[node] = (lat, lon)
    
    return coords

coords = generate_fake_coordinates(all_nodes)

for index, row in df.iterrows():
    vehicle = row["VehicleId"]
    seq = [int(x.strip()) for x in row["RouteSequence"].split("-")]

    first_node = seq[0]
    m = folium.Map(location=coords[first_node], zoom_start=11)

    # nodos
    for node in seq:
        folium.CircleMarker(
            location=coords[node],
            radius=6,
            popup=f"Municipio {node}",
            color="blue" if node != seq[0] else "red",
            fill=True
        ).add_to(m)

    # lineas
    route_coords = [coords[n] for n in seq]
    folium.PolyLine(route_coords, color="green", weight=4, opacity=0.7).add_to(m)

    # Guardar 
    output_file = f"mapa_vehiculo_{vehicle}.html"
    m.save(output_file)
    print(f"Mapa generado: {output_file}")



Mapa generado: mapa_vehiculo_1.html
Mapa generado: mapa_vehiculo_2.html
Mapa generado: mapa_vehiculo_8.html
